In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from emmv import emmv_scores
from sklearn.metrics import auc
from sklearn.preprocessing import StandardScaler
from pyod.models.iforest import IForest
from pyod.models.lof import LOF
from pyod.models.ocsvm import OCSVM
from pyod.models.knn import KNN
from pyod.models.auto_encoder import AutoEncoder

# **Data Prepocessing**

In [ ]:
# Load and concatenate data
df2018 = pd.read_csv('./2018Floor5.csv')
df2019 = pd.read_csv('./2019Floor5.csv')
df = pd.concat([df2018, df2019], ignore_index=True)

# Transform data
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
energy = df[['Date', 'z1_AC4(kW)']].copy()
energy.rename(columns={'z1_AC4(kW)': 'y'}, inplace=True)

# Set index and resample to hourly, summing and scaling by 1/60
energy.set_index('Date', inplace=True)
energy = energy.resample('d').sum() * (1/60)

# Extract features after resampling
energy['lag_7'] = energy['y'].shift(7)
# energy['hour'] = energy.index.hour
energy['day_of_week'] = energy.index.dayofweek
energy['is_weekend'] = energy['day_of_week'].isin([5, 6]).astype(int)
energy['rolling_mean'] = energy['y'].rolling(7).mean()
energy['rolling_std'] = energy['y'].rolling(7).std()

energy = energy.dropna()

In [ ]:
energy

# **Data Split**

In [ ]:
train_start = "2018-07-01"
train_end = "2018-09-30"
test_start = "2018-10-1"
test_end = "2019-12-31"
df_train = energy.loc[train_start:train_end]
df_test = energy.loc[test_start:test_end]

# **Pelatihan dan Pengujian**

In [ ]:
def apply_model(model, df_train, df_test):
    scaler = StandardScaler()
    df_train_scaled = scaler.fit_transform(df_train)
    df_test_scaled = scaler.transform(df_test)

    model.fit(df_train_scaled)
    pred = model.predict(df_test_scaled)
    anomalies = df_test[pred == 1]
    normal = df_test[pred == 0]
    score = model.decision_function(df_test_scaled)

    return model, pred, anomalies, normal, score, df_train_scaled, df_test_scaled

In [ ]:
def plot(df_test, score, pred, threshold):

    #threshold = np.min(score[pred == 1])
    print(f"Total data: {len(df_test)}")
    print(f"Anomali: {len(df_test[pred==1])}")
    print(f"Normal: {len(df_test[pred==0])}")

    print("\nDaftar waktu point anomaly:")
    anomaly_times = df_test.index[pred == 1]
    for i, time in enumerate(anomaly_times, 1):
        print(f"  - Anomali {i}: {time.strftime('%Y-%m-%d %H:%M:%S')}")

    fig, axs = plt.subplots(2, 1, figsize=(14, 8))

    axs[0].plot(df_test.index, df_test['y'], color='blue', label='Original data')
    axs[0].scatter(df_test.index[pred==1], df_test.y[pred==1], color='red', label='Anomalies')
    axs[0].legend()
    axs[0].grid()

    axs[1].plot(df_test.index, score, color='blue', label='Anomaly Score')
    axs[1].axhline(y=threshold, color='green', linestyle='--', label=f'Threshold ({threshold:.3e})')
    axs[1].scatter(df_test.index[pred == 1], score[pred == 1], color='red', label='Anomalies')
    axs[1].set_title('Anomaly Score')
    axs[1].legend()
    axs[1].grid()

    plt.tight_layout()
    plt.show()


In [ ]:
def get_collective_anomaly_ranges(preds, timestamps):
    ranges = []
    start = None
    for i, val in enumerate(preds):
        if val == 1 and start is None:
            start = timestamps[i]
        elif val == 0 and start is not None:
            #end = timestamps[i]: kalo mau "anomali berlangsung sampai data kembali normal"
            #end = timestamps[i-1]: kalo mau rangenya hanya mencakup data yang memang anomali
            end = timestamps[i]
            ranges.append((start, end))
            start = None
    if start is not None:
        ranges.append((start, timestamps[-1]))
    return ranges

def plot_collective(df_test, score, pred, threshold):
    print(f"Total data: {len(df_test)}")
    print(f"Anomali: {len(df_test[pred==1])}")
    print(f"Normal: {len(df_test[pred==0])}")

    fig, axs = plt.subplots(2, 1, figsize=(16, 8))

    axs[0].plot(df_test.index, df_test['y'], color='blue', label='Original Data')

    ranges = get_collective_anomaly_ranges(pred, df_test.index)
    print("\nDaftar kolektif anomali:")
    for i, (start, end) in enumerate(ranges, 1):
        print(f"  - Anomali {i}: {start.strftime('%Y-%m-%d %H:%M')} → {end.strftime('%Y-%m-%d %H:%M')}")
        axs[0].axvspan(start, end, color='red', alpha=0.3)

    axs[0].set_title('Data')
    axs[0].legend()
    axs[0].grid()

    # Plot skor anomali
    axs[1].plot(df_test.index, score, color='blue', label='Anomaly Score')
    axs[1].axhline(y=threshold, color='green', linestyle='--', label=f'Threshold ({threshold:.3e})')
    axs[1].scatter(df_test.index[pred == 1], score[pred == 1], color='red', s=10, label='Anomaly Points')
    for start, end in ranges:
        axs[1].axvspan(start, end, color='red', alpha=0.3)
        
    axs[1].set_title('Anomaly Score')
    axs[1].legend()
    axs[1].grid()

    plt.tight_layout()
    plt.show()


## **Isolation Forest**

In [ ]:
if_model, if_pred, if_anomalies, if_normal, if_score, train_scaled, test_scaled = apply_model(IForest(contamination=0.01, random_state=1), df_train, df_test)

In [ ]:
plot(df_train, if_model.decision_scores_, if_model.labels_, if_model.threshold_)

In [ ]:
plot(df_test, if_score, if_pred, if_model.threshold_)

In [ ]:
plot_collective(df_test, if_score, if_pred, if_model.threshold_)

## **LOF**

In [ ]:
lof_model, lof_pred, lof_anomalies, lof_normal, lof_score, train_scaled, test_scaled = apply_model(LOF(contamination=0.01, novelty=True), df_train, df_test)

In [ ]:
plot(df_train, lof_model.decision_scores_, lof_model.labels_, lof_model.threshold_)

In [ ]:
plot(df_test, lof_score, lof_pred, lof_model.threshold_)

In [ ]:
plot_collective(df_test, lof_score, lof_pred, lof_model.threshold_)

## **AutoEncoder**

In [ ]:
ae_model, ae_pred, ae_anomalies, ae_normal, ae_score, train_scaled, test_scaled = apply_model(AutoEncoder(contamination = 0.01), df_train, df_test)

In [ ]:
plot(df_train, ae_model.decision_scores_, ae_model.labels_, ae_model.threshold_)

In [ ]:
plot(df_test, ae_score, ae_pred, ae_model.threshold_)

In [ ]:
plot_collective(df_test, ae_score, ae_pred, ae_model.threshold_)

In [ ]:
joblib.dump(ae_model, 'ae_model.pkl')

## **OneClassSVM**

In [ ]:
ocsvm_model, ocsvm_pred, ocsvm_anomalies, ocsvm_normal, ocsvm_score, train_scaled, test_scaled = apply_model(OCSVM(contamination=0.01), df_train, df_test)

In [ ]:
plot(df_train, ocsvm_model.decision_scores_, ocsvm_model.labels_, ocsvm_model.threshold_)

In [ ]:
plot(df_test, ocsvm_score, ocsvm_pred, ocsvm_model.threshold_)

In [ ]:
plot_collective(df_test, ocsvm_score, ocsvm_pred, ocsvm_model.threshold_)

## **KNN**

In [ ]:
knn_model, knn_pred, knn_anomalies, knn_normal, knn_score, train_scaled, test_scaled = apply_model(KNN(contamination=0.01), df_train, df_test)

In [ ]:
plot(df_train, knn_model.decision_scores_, knn_model.labels_, knn_model.threshold_)

In [ ]:
plot(df_test, knn_score, knn_pred, knn_model.threshold_)

In [ ]:
plot_collective(df_test, knn_score, knn_pred, knn_model.threshold_)

# **EMMV**

In [ ]:
n_generated = 10000
alpha_min = 0.005
alpha_max = 0.999
t_max = 0.9


In [ ]:
# hitung exess mass
def em(t, t_max, volume_support, s_unif, s_X, n_generated):
    # inisialisasi array untuk simpan nilai em
    EM_t = np.zeros(t.shape[0]) 
    # jumlah data dalam data test
    n_samples = s_X.shape[0]
    # nilai unik dari skor anomali data test
    s_X_unique = np.unique(s_X)
    EM_t[0] = 1.
    for u in s_X_unique:
        # hitung excess mass dengan mengurangi massa aktual dan massa uniform yang dikali penalti (t)
        EM_t = np.maximum(EM_t, 1. / n_samples * (s_X > u).sum() -
                          t * (s_unif > u).sum() / n_generated * volume_support)
    amax = np.argmax(EM_t <= t_max) + 1
    if amax == 1:
        print('\nFailed to achieve t_max\n')
        amax = -1
    AUC = auc(t[:amax], EM_t[:amax])
    return AUC, EM_t, amax

# hitung mass volume
def mv(axis_alpha, volume_support, s_unif, s_X, n_generated):
    # jumlah data dalam data test
    n_samples = s_X.shape[0]
    # skor anomalinya diurutin dari kecil ke besar
    s_X_argsort = s_X.argsort()
    mass = 0
    cpt = 0
    # ambang batas awal diambil skor anomali yang terbesar
    u = s_X[s_X_argsort[-1]]
    # inisialisasi array untuk simpan nilai mv
    mv = np.zeros(axis_alpha.shape[0])
    for i in range(axis_alpha.shape[0]):
        while mass < axis_alpha[i]:
            cpt += 1
            u = s_X[s_X_argsort[-cpt]]
            mass = 1. / n_samples * cpt
        # hitung volume berdasarkan proporsi data uniform yang lebih dari sm dengan ambang (u)
        mv[i] = float((s_unif >= u).sum()) / n_generated * volume_support
    return auc(axis_alpha, mv), mv

def compute_em_mv(model, df_train_scaled, df_test_scaled, score, n_generated, alpha_min, alpha_max, t_max):
    n_samples, n_features = df_test_scaled.shape

    X_train_ = df_train_scaled
    X_ = df_test_scaled
    # skor anomali data test, dikasi min soalnya kebalikan dengan scikit learn
    s_X = -score  
    
    # nilai min setiap fitur dari data test
    lim_inf = X_.min(axis=0)
    # nilai maks setiap fitur dari data test
    lim_sup = X_.max(axis=0)
    #hitung volume support dengan mengalikan seluruh selisih nilai maks dan min setiap fitur
    volume_support = (lim_sup - lim_inf).prod()
    # array skala pinalti untuk menentukan seberapa ketat mendeteksi anomali
    t = np.arange(0, 100 / volume_support, 0.001 / volume_support)
    # array proporsi massa untuk mv
    axis_alpha = np.arange(alpha_min, alpha_max, 0.001)
    # buat data acak uniform dalam rentang fitur pengujian
    unif = np.random.uniform(lim_inf, lim_sup, size=(n_generated, n_features))
    s_unif = -model.decision_function(unif)

    em_auc, em_t, amax = em(t, t_max, volume_support, s_unif, s_X, n_generated)
    mv_auc, mv_curve = mv(axis_alpha, volume_support, s_unif, s_X, n_generated)

    plt.figure(figsize=(12, 5))
    plt.subplot(121)
    plt.plot(t[:amax], em_t[:amax], label=f'em_score = {em_auc:.3e}')
    plt.ylim([-0.05, 1.05])
    plt.xlabel('t')
    plt.ylabel('EM(t)')
    plt.title('Excess-Mass Curve')
    plt.legend(loc="lower right")
    plt.grid()
    
    plt.subplot(122)
    plt.plot(axis_alpha, mv_curve, label=f'mv_score = {mv_auc:.3e}')
    plt.xlabel('alpha')
    plt.ylabel('MV(alpha)')
    plt.title('Mass-Volume Curve')
    plt.legend(loc="upper left")

    plt.grid()
    plt.tight_layout()
    plt.show()
    return em_auc, mv_auc

In [ ]:
em_value_if, mv_value_if = compute_em_mv(if_model, train_scaled, test_scaled, if_score, n_generated, alpha_min, alpha_max, t_max)
print(f"EM: {em_value_if:.4e}, MV: {mv_value_if:.4e}")

In [ ]:
em_value_lof, mv_value_lof = compute_em_mv(lof_model, train_scaled, test_scaled, lof_score, n_generated, alpha_min, alpha_max, t_max)
print(f"EM: {em_value_lof:.4e}, MV: {mv_value_lof:.4e}")

In [ ]:
em_value_ae, mv_value_ae = compute_em_mv(ae_model, train_scaled, test_scaled, ae_score, n_generated, alpha_min, alpha_max, t_max)
print(f"EM: {em_value_ae:.4e}, MV: {mv_value_ae:.4e}")

In [ ]:
em_value_ocsvm, mv_value_ocsvm = compute_em_mv(ocsvm_model, train_scaled, test_scaled, ocsvm_score, n_generated, alpha_min, alpha_max, t_max)
print(f"EM: {em_value_ocsvm:.4e}, MV: {mv_value_ocsvm:.4e}")

In [ ]:
em_value_knn, mv_value_knn = compute_em_mv(knn_model, train_scaled, test_scaled, knn_score, n_generated, alpha_min, alpha_max, t_max)
print(f"EM: {em_value_knn:.4e}, MV: {mv_value_knn:.4e}")

In [ ]:
emmv_score = pd.DataFrame({
    'model': ['Isolation Forest', 'Local Outlier Factor', 'Auto Encoder', 'OneClass SVM', 'KNN'], 
    'Excess Mass': [em_value_if, em_value_lof, em_value_ae, em_value_ocsvm, em_value_knn],
    'Mass Volume': [mv_value_if, mv_value_lof, mv_value_ae, mv_value_ocsvm, mv_value_knn],
})
emmv_score